In [ ]:

import os

from neuralforecast.models.patchtst import PatchTST
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import pandas as pd
from neuralforecast.core import NeuralForecast
from neuralforecast.models.mixmodel_rescale_deco import MIXModel
from neuralforecast.losses.pytorch import MAE, MSE
from neuralforecast.losses.numpy import mae, mse
# from datasetsforecast.long_horizon import LongHorizon, LongHorizonInfo
from datasetsforecast.long_horizon2 import LongHorizon2, LongHorizon2Info

import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
epochs = 300
def timedeter(modelname,horizon,dataset):
  if modelname == 'MIXModel':
    model = MIXModel(
        h = horizon,
        input_size = 512,
        decomp_kernel_size = 25, 
        transformer_input_size = 192,
        n_blocks = [1, 1, 1],
        mlp_units = 3 * [[256, 256]],
        n_pool_kernel_size = [16, 8, 1],
        n_freq_downsample = [16, 8, 1],
        pooling_mode = "MaxPool1d",
        interpolation_mode = "linear",
        dropout_prob_theta=0.2,
        mlp_activation = "ReLU",
        
        encoder_layers = 3,
        n_heads = 4,
        hidden_size = 64,
        linear_hidden_size = 128,
        dropout = 0.2,
        fc_dropout = 0.2,
        head_dropout = 0.0,
        attn_dropout = 0.0,
        patch_len = 16,
        stride = 8,
        revin = True,
        revin_affine = False,
        revin_subtract_last = True,
        activation = "gelu",
        res_attention = True,
        batch_normalization = False,
        learn_pos_embed = True,
        loss=MAE(),
        valid_loss=MAE(),
        max_steps = epochs,
        learning_rate = 1e-4,
        num_lr_decays = -1,
        early_stop_patience_steps = 3,
        val_check_steps = 20,
        batch_size = 128,
        valid_batch_size = None,
        windows_batch_size = 1024, 
        inference_windows_batch_size = 1024,
        step_size = 2, 
        scaler_type = "identity",
        random_seed = 2023,
        num_workers_loader = 0, # 取决于 cpu 数量
        stat_exog_list=None,
        hist_exog_list=None,
        futr_exog_list=None,
        exclude_insample_y=False,
        drop_last_loader = False,
)
  elif modelname == 'PatchTST':
    model = PatchTST(
        h = horizon,
        input_size = 336,
        
        encoder_layers = 3,
        n_heads = 8,
        hidden_size = 64,
        linear_hidden_size = 256,
        dropout = 0.2,
        fc_dropout = 0.2,
        head_dropout = 0.0,
        attn_dropout = 0.0,
        patch_len = 16,
        stride = 8,
        revin = True,
        revin_affine = False,
        revin_subtract_last = True,
        activation = "gelu",
        res_attention = True,
        batch_normalization = False,
        learn_pos_embed = True,
        loss=MAE(),
        valid_loss=MAE(),
        max_steps = epochs,
        learning_rate = 1e-3,
        num_lr_decays = -1,
        early_stop_patience_steps = 3,
        val_check_steps = 10,
        batch_size = 32,
        valid_batch_size = None,
        windows_batch_size = 1024, 
        inference_windows_batch_size = 1024,
        step_size = 2, 
        scaler_type = "standard",
        random_seed = 2023,
        num_workers_loader = 0, # 取决于 cpu 数量
        stat_exog_list=None,
        hist_exog_list=None,
        futr_exog_list=None,
        exclude_insample_y=False,
        drop_last_loader = False,
)

  Y_df = LongHorizon2.load(directory="./data/", group=dataset)
  Y_df["ds"] = pd.to_datetime(Y_df["ds"])
  n_time = LongHorizon2Info[dataset].n_time
  freq = LongHorizon2Info[dataset].freq
  val_size = LongHorizon2Info[dataset].val_size
  test_size = LongHorizon2Info[dataset].test_size
  Y_df.drop(columns=["index"], inplace=True)

  # fit & test model
  nf = NeuralForecast(models=[model], freq=freq)
  Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size,step_size=1, n_windows=None)
  # calculate matrics
  y_true = Y_hat_df.y.values
  y_hat = Y_hat_df[modelname].values
  n_series = len(Y_hat_df.unique_id.unique())
  y_true = y_true.reshape(n_series, -1, horizon)
  y_hat = y_hat.reshape(n_series, -1, horizon)
  return mae(y_hat, y_true), mse(y_hat, y_true)




models = ['MIXModel', 'PatchTST']
ETTh1_horizon = [24,24*7,24*7*2,24*30]
ETTm1_horizon = [96,96*7,96*7*2]

result = pd.DataFrame(columns=['model','dataset','horizon','mae','mse'])

for model in models:
  for horizon in ETTh1_horizon:
    dataset = 'ETTh1'
    _mae , _mse = timedeter(model,horizon,dataset)
    result = result._append({'model':model,'dataset':dataset,'horizon':horizon,'mae':_mae,'mse':_mse},ignore_index=True)

for model in models:
  for horizon in ETTm1_horizon:
    dataset = 'ETTm1'
    _mae,_mse = timedeter(model,horizon,dataset)
    result = result._append({'model':model,'dataset':dataset,'horizon':horizon,'mae':_mae,'mse':_mse},ignore_index=True)
    
result.to_csv('./data/time_determind.csv')

In [20]:
result

,model,dataset,horizon,mae,mse
0,MIXModel,ETTh1,24,0.333124,0.268059
1,MIXModel,ETTh1,168,0.394871,0.361756
2,MIXModel,ETTh1,336,0.414464,0.397811
3,MIXModel,ETTh1,720,0.429437,0.386717
4,PatchTST,ETTh1,24,0.343953,0.277722
5,PatchTST,ETTh1,168,0.391764,0.353621
6,PatchTST,ETTh1,336,0.420193,0.395108
7,PatchTST,ETTh1,720,0.457165,0.429300
8,MIXModel,ETTm1,96,0.318104,0.253238
9,MIXModel,ETTm1,672,0.390404,0.363776
